# Μέρος Β1.i: Ταξινόμηση Νομικών Εγγράφων με Support Vector Machines (SVM) 

Αυτό το notebook αφορά την υλοποίηση και αξιολόγηση μοντέλων Support Vector Machines (SVM) για την ταξινόμηση ελληνικών νομικών εγγράφων, χρησιμοποιώντας αναπαραστάσεις κειμένου Bag-of-Words (BoW) και TF-IDF. Η εργασία αυτή αποτελεί μέρος του ερωτήματος Β1.i της εξαμηνιαίας εργασίας.

**Επεξήγηση Κελιού:**
Εισαγωγή στο notebook: SVM με BoW/TF-IDF για ταξινόμηση νομικών κειμένων (Β1.i).

## 1. Τεχνική Ανάλυση Αλγορίθμων

### 1.1. Μηχανές Υποστήριξης Διανυσμάτων (Support Vector Machines - SVM)

Οι SVMs βρίσκουν ένα βέλτιστο υπερεπίπεδο που διαχωρίζει κλάσεις μεγιστοποιώντας το περιθώριο μεταξύ τους. Χρησιμοποιούνται για ταξινόμηση.

**Βασικά Σημεία:**
- **Υπερεπίπεδο & Περιθώριο:** Στόχος είναι το υπερεπίπεδο με το μεγαλύτερο περιθώριο.
- **Διανύσματα Υποστήριξης:** Σημεία δεδομένων που καθορίζουν το υπερεπίπεδο.
- **Πυρήνες (Kernels):** Για μη γραμμικά διαχωρίσιμα δεδομένα (π.χ., 'linear', 'rbf', 'poly'). Χρησιμοποιήθηκε 'linear'.
- **Παράμετρος `C`:** Ελέγχει τον συμβιβασμό μεταξύ μεγιστοποίησης περιθωρίου και ελαχιστοποίησης σφαλμάτων ταξινόμησης. Χρησιμοποιήθηκε `C=1.0`.
- **Πλεονεκτήματα:** Αποτελεσματικά σε υψηλές διαστάσεις, αποδοτικά ως προς τη μνήμη.
- **Μειονεκτήματα:** Αργά σε μεγάλα datasets, ευαίσθητα στην επιλογή πυρήνα/παραμέτρων.

**Επεξήγηση Κελιού:**
Συνοπτική τεχνική ανάλυση των SVM: βασική ιδέα, πυρήνες, παράμετρος C, πλεονεκτήματα και μειονεκτήματα.

### 1.2. Αναπαράσταση Κειμένου: Σάκος Λέξεων (Bag-of-Words - BoW)

Το BoW αναπαριστά κείμενο ως ένα διάνυσμα συχνοτήτων λέξεων από ένα λεξιλόγιο.

**Βασικά Σημεία:**
- **Διαδικασία:** Δημιουργία λεξιλογίου, διανυσματοποίηση εγγράφων με βάση τις συχνότητες λέξεων.
- **Παράμετροι (`CountVectorizer`):** `max_features` (μέγεθος λεξιλογίου, εδώ 5000), `stop_words`, `ngram_range`.
- **Πλεονεκτήματα:** Απλότητα, αποδοτικότητα.
- **Μειονεκτήματα:** Απώλεια σειράς λέξεων και σημασιολογίας, υψηλή διαστατικότητα/αραιότητα.

**Επεξήγηση Κελιού:**
Συνοπτική τεχνική ανάλυση του BoW: διαδικασία, παράμετροι, πλεονεκτήματα και μειονεκτήματα.

### 1.3. Αναπαράσταση Κειμένου: TF-IDF (Term Frequency-Inverse Document Frequency)

Το TF-IDF δίνει βάρος στις λέξεις ανάλογα με τη συχνότητά τους σε ένα έγγραφο (TF) και την αντιστρόφως ανάλογη συχνότητά τους στο σύνολο των εγγράφων (IDF).

**Βασικά Σημεία:**
- **TF (Term Frequency):** Συχνότητα όρου στο έγγραφο.
- **IDF (Inverse Document Frequency):** Σημαντικότητα όρου στο σώμα κειμένων.
- **TF-IDF Score:** `TF * IDF`. Υψηλό για όρους συχνούς στο έγγραφο αλλά σπάνιους γενικά.
- **Παράμετροι (`TfidfVectorizer`):** Παρόμοιες με `CountVectorizer` (π.χ. `max_features=5000`), `use_idf`, `smooth_idf`, `norm`.
- **Πλεονεκτήματα:** Μειώνει βάρος κοινών λέξεων, τονίζει χαρακτηριστικές λέξεις.
- **Μειονεκτήματα:** Αγνοεί σειρά/σημασιολογία, αραιότητα.

**Επεξήγηση Κελιού:**
Συνοπτική τεχνική ανάλυση του TF-IDF: συστατικά, score, παράμετροι, πλεονεκτήματα και μειονεκτήματα.

### 1.4. Συνδυασμός SVM με BoW/TF-IDF

Η διαδικασία συνδυασμού περιλαμβάνει:
1.  **Προεπεξεργασία Κειμένου.**
2.  **Εξαγωγή Χαρακτηριστικών** με BoW ή TF-IDF.
3.  **Εκπαίδευση SVM** με τα διανύσματα χαρακτηριστικών.

**Οφέλη:** Τα SVMs είναι αποτελεσματικά σε χώρους υψηλών διαστάσεων που παράγονται από BoW/TF-IDF. Το TF-IDF μπορεί να βελτιώσει την ποιότητα των χαρακτηριστικών.

**Επεξήγηση Κελιού:**
Περιγραφή του συνδυασμού SVM με BoW/TF-IDF: βήματα και οφέλη.

## 2. Υλοποίηση και Πειράματα

Στα παρακάτω κελιά κώδικα, θα ρυθμίσουμε και θα εκτελέσουμε τα πειράματα ταξινόμησης χρησιμοποιώντας SVM με BoW και TF-IDF. Θα χρησιμοποιήσουμε τις συναρτήσεις από το αρχείο `utils.py`.

**Επεξήγηση Κελιού:**
Έναρξη της ενότητας υλοποίησης και πειραμάτων.

In [ ]:
import sys
import os
import numpy as np

current_notebook_dir = os.getcwd()
parent_dir = os.path.dirname(current_notebook_dir)
if parent_dir not in sys.path:
    sys.path.insert(0, parent_dir)

try:
    from utils import (
        load_and_preprocess_data,
        run_experiment,
        script_execution_timer
    )
    print("Successfully imported from utils.py")
except ImportError as e:
    print(f"Error importing from utils.py: {e}")
    print(f"Please ensure utils.py is in the correct path: {parent_dir} or that the notebook's parent directory is correctly identified.")

from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.svm import SVC

**Επεξήγηση Κελιού:**
Εισαγωγή βιβλιοθηκών και ρύθμιση path για το `utils.py`.

### 2.1. Ρυθμίσεις Πειράματος

Παρακαλώ επιλέξτε τις παραμέτρους για το πείραμα. Μπορείτε να αλλάξετε τις τιμές των μεταβλητών στο παρακάτω κελί κώδικα.

**Επεξήγηση Κελιού:**
Ενότητα διαμόρφωσης παραμέτρων πειράματος.

In [ ]:
# --- Configuration ---
DATASET_CONFIG = "volume" 
FEATURE_METHOD = "BoW" # or "TF-IDF"
SUBSET_PERCENTAGE = 0.1 
N_SPLITS_CV = 1 
RANDOM_STATE = 42
VECTORIZER_MAX_FEATURES = 5000 
SINGLE_SPLIT_TEST_SIZE = 0.2 
SAVE_TRAINED_FEATURE_MODELS = True
LOAD_TRAINED_FEATURE_MODELS_IF_EXIST = True
# --- End Configuration ---

**Επεξήγηση Κελιού:**
Παράμετροι διαμόρφωσης για το πείραμα (dataset, feature method, split, vectorizer).

In [ ]:
@script_execution_timer
def run_svm_experiment(dataset_config, feature_method, subset_percentage, n_splits_cv, random_state, vectorizer_max_features, single_split_test_size, save_models, load_models):
    model_name_script = "SVM"
    feature_method_name = feature_method
    base_run_id = f"{model_name_script}_{feature_method_name.replace('-', '')}_{dataset_config}" 
    print(f"Starting {model_name_script} with {feature_method_name} for '{dataset_config}' config...")
    
    if feature_method_name == "BoW":
        feature_config = {
            'method': 'sklearn_vectorizer',
            'class': CountVectorizer,
            'params': {'max_features': vectorizer_max_features, 'stop_words': None}
        }
    elif feature_method_name == "TF-IDF":
        feature_config = {
            'method': 'sklearn_vectorizer',
            'class': TfidfVectorizer,
            'params': {'max_features': vectorizer_max_features, 'stop_words': None}
        }
    else:
        print(f"ERROR: Unknown feature_method: {feature_method_name}")
        return
    
    model_class = SVC
    model_init_params = {'kernel': 'linear', 'probability': True, 'random_state': random_state, 'C': 1.0}

    script_dir = os.getcwd()
    experiment_output_base_dir = os.path.join(script_dir, "outputs", base_run_id)
    reports_output_dir = os.path.join(experiment_output_base_dir, "reports")
    feature_models_output_dir = os.path.join(experiment_output_base_dir, "feature_models")
    os.makedirs(reports_output_dir, exist_ok=True)
    os.makedirs(feature_models_output_dir, exist_ok=True)
    print(f"INFO: Experiment outputs will be saved in '{experiment_output_base_dir}'")

    if 0 < subset_percentage < 1.0:
        print(f"INFO: Using a {subset_percentage*100:.0f}% subset of the data.")
    else:
        print("INFO: Using all available data (after filtering).")
    
    if n_splits_cv == 1:
        print(f"INFO: Performing a single train/test split (test_size={single_split_test_size}).")
    elif n_splits_cv > 1:
        print(f"INFO: Performing {n_splits_cv}-Fold Cross-Validation.")
    else:
        print("ERROR: N_SPLITS_CV must be >= 1.")
        return

    print(f"INFO: Saving of trained feature models is {'ENABLED' if save_models else 'DISABLED'}.")
    print(f"INFO: Loading of existing feature models is {'ENABLED' if load_models else 'DISABLED'}.")

    texts_proc, labels_proc, unique_labels_proc, unique_labels_proc_str = load_and_preprocess_data(
        dataset_config, subset_percentage, random_state
    )

    if texts_proc is None or len(texts_proc) == 0:
        print("Failed to load or preprocess data. Exiting.")
        return

    success = run_experiment(
        texts_to_process=texts_proc,
        labels_to_process=labels_proc,
        unique_labels_to_process=unique_labels_proc,
        unique_labels_to_process_str=unique_labels_proc_str,
        n_splits=n_splits_cv,
        feature_config=feature_config,
        model_class=model_class,
        model_init_params=model_init_params,
        base_run_identifier=base_run_id, 
        reports_output_dir=reports_output_dir,
        feature_models_output_dir=feature_models_output_dir,
        random_state=random_state,
        save_trained_features=save_models,
        load_trained_features_if_exist=load_models,
        single_split_test_size=single_split_test_size
    )

    if success:
        print(f"\nExperiment '{base_run_id}' completed successfully.")
    else:
        print(f"\nExperiment '{base_run_id}' encountered errors.")

    print(f"\n{model_name_script} with {feature_method_name} script finished for {dataset_config} dataset.")

**Επεξήγηση Κελιού:**
Συνάρτηση `run_svm_experiment` που ενσωματώνει τη λογική εκτέλεσης του πειράματος.

### 2.2. Εκτέλεση Πειράματος

Το παρακάτω κελί θα εκτελέσει το πείραμα με τις ρυθμίσεις που ορίστηκαν παραπάνω.

**Επεξήγηση Κελιού:**
Εισαγωγή στο κελί εκτέλεσης του πειράματος.

In [ ]:
run_svm_experiment(
    dataset_config=DATASET_CONFIG,
    feature_method=FEATURE_METHOD,
    subset_percentage=SUBSET_PERCENTAGE,
    n_splits_cv=N_SPLITS_CV,
    random_state=RANDOM_STATE,
    vectorizer_max_features=VECTORIZER_MAX_FEATURES,
    single_split_test_size=SINGLE_SPLIT_TEST_SIZE,
    save_models=SAVE_TRAINED_FEATURE_MODELS,
    load_models=LOAD_TRAINED_FEATURE_MODELS_IF_EXIST
)

**Επεξήγηση Κελιού:**
Εκτέλεση του πειράματος με τις καθορισμένες παραμέτρους.

## 3. Αποτελέσματα και Σχολιασμός

Τα αναλυτικά αποτελέσματα (classification reports) για κάθε εκτέλεση αποθηκεύονται στον κατάλογο `outputs/[MODEL_FEATURE_CONFIG]/reports/`. 

Για παράδειγμα, αν εκτελέσατε SVM με BoW για το `subject`:
`merosB1/i/outputs/SVM_BoW_subject/reports/`

Τα αποτελέσματα που παρουσιάζονται στην επιστημονική αναφορά (`B1.md`) προέρχονται από εκτελέσεις αυτού του κώδικα.

**Επεξήγηση Κελιού:**
Πληροφορίες για την εύρεση των αποτελεσμάτων και σύνδεση με την επιστημονική αναφορά.